In [21]:
from decouple import config, AutoConfig
import os
import requests
os.chdir("../")
print("Current Working Directory:", os.getcwd())
print("Files in Current Directory:", os.listdir(os.getcwd()))
config = AutoConfig(search_path="/home/harry/StockTradingbot") 

Current Working Directory: /
Files in Current Directory: ['media', 'dev', 'sys', 'root', 'bin', 'lost+found', 'run', 'boot', 'usr', 'etc', 'var', 'opt', 'mnt', 'srv', 'lib', 'snap', 'home', 'Docker', 'init', 'lib32', 'proc', 'libx32', 'sbin', 'lib64', 'tmp']


In [22]:
ALPHA_VANTAGE_API_KEY =  config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

In [23]:
ALPHA_VANTAGE_API_KEY

'SNCFS0TQWEYDXJLH'

In [18]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
api_key = ALPHA_VANTAGE_API_KEY  # Replace with your actual API key
params = {
    "function": "TIME_SERIES_INTRADAY",
    "ticker": "AAPL",
    "interval": "1min",
    "apikey": ALPHA_VANTAGE_API_KEY
}

url = 'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval={interval}&apikey={apikey}'.format(
    **params
)
r = requests.get(url)
data = r.json()

In [40]:
dataset_key = [x for x in list(data.keys()) if not x.lower() == "meta data"][0]
dataset_key

'Time Series (1min)'

In [41]:
results = data[dataset_key] #dict
#for timestamp_str in results.keys():
#    print(timestamp_str, results.get(timestamp_str))

In [42]:
timestamp_str = list(results.keys())[0]
timestamp_str

'2025-01-03 19:59:00'

In [43]:
result_value = results.get(timestamp_str)
result_value

{'1. open': '243.3600',
 '2. high': '243.4000',
 '3. low': '243.3300',
 '4. close': '243.3500',
 '5. volume': '661'}

In [44]:
from datetime import datetime
from decimal import Decimal
import pytz 

def transfor_alpha_vantage_result(timestamp_str, result):
    timestamp_format = '%Y-%m-%d %H:%M:%S'
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc
    timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)
    # unix_timestamp = result.get('t') / 1000.0
    # utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone('UTC'))
    return{
         'open_price': Decimal(result['1. open']),
         'close_price': Decimal(result['3. low']),
         'high_price': Decimal(result['2. high']),
         'low_price': Decimal(result['3. low']),
         'volume': int(result['5. volume']),
         'number_of_trades': None,
         'volume_weighted_average': None,
         'time': timestamp,
}

In [45]:
transfor_alpha_vantage_result(timestamp_str, result_value)

{'open_price': Decimal('243.3600'),
 'close_price': Decimal('243.3300'),
 'high_price': Decimal('243.4000'),
 'low_price': Decimal('243.3300'),
 'volume': 661,
 'number_of_trades': None,
 'volume_weighted_average': None,
 'time': datetime.datetime(2025, 1, 4, 0, 59, tzinfo=<UTC>)}

In [46]:
for timestamp_str in results.keys():
    new_data = transfor_alpha_vantage_result(timestamp_str, results.get(timestamp_str))
    print(new_data)

{'open_price': Decimal('243.3600'), 'close_price': Decimal('243.3300'), 'high_price': Decimal('243.4000'), 'low_price': Decimal('243.3300'), 'volume': 661, 'number_of_trades': None, 'volume_weighted_average': None, 'time': datetime.datetime(2025, 1, 4, 0, 59, tzinfo=<UTC>)}
{'open_price': Decimal('243.3500'), 'close_price': Decimal('243.3000'), 'high_price': Decimal('243.3600'), 'low_price': Decimal('243.3000'), 'volume': 169, 'number_of_trades': None, 'volume_weighted_average': None, 'time': datetime.datetime(2025, 1, 4, 0, 58, tzinfo=<UTC>)}
{'open_price': Decimal('243.3000'), 'close_price': Decimal('243.3000'), 'high_price': Decimal('243.3500'), 'low_price': Decimal('243.3000'), 'volume': 604, 'number_of_trades': None, 'volume_weighted_average': None, 'time': datetime.datetime(2025, 1, 4, 0, 57, tzinfo=<UTC>)}
{'open_price': Decimal('243.3200'), 'close_price': Decimal('243.3000'), 'high_price': Decimal('243.3500'), 'low_price': Decimal('243.3000'), 'volume': 2270, 'number_of_trades'